In [16]:
start = 39421
base_format = "https://www.ptt.cc/bbs/Gossiping/index{index}.html"
page = 10000

In [17]:
import requests
from bs4 import BeautifulSoup

targets = []
for p in range(page):
    response = requests.get(base_format.format(index=str(start - p)),
                            cookies={"over18":"1"})
    html = BeautifulSoup(response.text, "lxml")
    entries = html.find_all("div", class_="r-ent")
    for e in entries:
        t = e.find("div", class_="title").find("a")
        if t is not None:
            title = t.text
            if title.startswith("[") and not title.startswith("[公告]"):
                targets.append((title, t["href"]))
                
print("總共:", len(targets), "篇文章")
print("前十篇:", targets[:10])

總共: 128967 篇文章
前十篇: [('[問卦] ubereat人員素質比較好一些', '/bbs/Gossiping/M.1570852908.A.FE1.html'), ('[爆卦] 蔡依林演唱會門票1分鐘6W張完售', '/bbs/Gossiping/M.1570853001.A.A93.html'), ('[問卦] 要如何拯救離家出走的少女？', '/bbs/Gossiping/M.1570853006.A.1C9.html'), ('[爆卦] 美國公司賣催淚彈給港警', '/bbs/Gossiping/M.1570853009.A.03D.html'), ('[問卦] 國共特工香港大亂鬥', '/bbs/Gossiping/M.1570853129.A.083.html'), ("[問卦] 女兒長醬子 484養成成功惹? o'_'o", '/bbs/Gossiping/M.1570853139.A.C48.html'), ('[新聞] 打破沉默！暴雪：聰哥禁賽降為六個月、', '/bbs/Gossiping/M.1570853301.A.BE9.html'), ('[問卦] 星野源演唱會購票 要回答問題', '/bbs/Gossiping/M.1570853348.A.10D.html'), ('[問卦] 阿飛出來幹嘛啦 LUL', '/bbs/Gossiping/M.1570853368.A.FD7.html'), ('[問卦] 正妹發傳單刻意避開我', '/bbs/Gossiping/M.1570853464.A.3E9.html')]


In [63]:
def save_article(article, dn):
    article_base = "https://www.ptt.cc"
    article_url = article_base + article
    response = requests.get(article_url, cookies={"over18":"1"})
    html = BeautifulSoup(response.text, "lxml")

    main = html.find("div", id="main-content")
    metas = main.find_all("span", class_="article-meta-value")
    user_id = metas[0].text.split(" ")[0]
    title = metas[2].text
    pattern = re.compile(r"\[.+\]")
    article_type = (re.match(pattern, title)
               .group()
               .replace("[", "")
               .replace("]", ""))
    article_title = re.split(pattern, title)[-1].lstrip(" ")
    article_time = str(datetime.strptime(metas[3].text, 
                                         "%a %b %d %H:%M:%S %Y"))

    pushes = main.find_all("div", class_="push")
    article_push = []
    article_score = 0

    for m in main.find_all("div", class_="article-metaline"):
        m.extract()
    for m in main.find_all("div", class_="article-metaline-right"):
        m.extract()    

    for p in pushes:
        tag = p.find("span", class_="push-tag")
        if "推" in tag.text:
            push_score = 1
            article_score = article_score + 1
        elif "噓" in tag.text:
            push_score = -1
            article_score = article_score - 1
        else:
            push_score = 0
        push_id = p.find("span", class_="push-userid").text
        push_content = p.find("span", class_="push-content").text
        article_push.append({"type":push_score,
                             "id":push_id,
                             "content":push_content})
        p.extract()

    ip = None
    for f in main.find_all("span", class_="f2"):
        if "發信站:" in f.text:
            pat = re.compile("[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+")
            ip = re.search(pat, f.text).group()
            f.extract()
        elif "文章網址:" in f.text:
            f.extract()

    data = {
        "ptt_id":user_id,
        "post_url":article_url,
        "post_type":article_type,
        "post_title":article_title,
        "post_time":article_time,
        "post_score":article_score,
        "post_pushes":article_push,
        "post_ip":ip,
        "post_content":main.text
    }

    fn = article_url.split("/")[-1].split(".html")[0]
    with open(os.path.join(dn, fn + ".json"), "w", encoding="utf-8") as f:
        json.dump(data, f)

In [ ]:
import re
from datetime import datetime
import json
import os

dn = "pttdata"
if not os.path.exists(dn):
    os.makedirs(dn)

for t in targets:
    try:
        save_article(t[1], dn)
    except Exception as e:
        print(e)

[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'find_all'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'find_all'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
list index out of range
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'find_all'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
list index out of range
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.F

[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'text'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bbs/Gossiping/M.1570852908.A.FE1.html
'NoneType' object has no attribute 'group'
[Error] https://www.ptt.cc/bb